In [1]:
import pandas as pd
import nltk
import os
import numpy as np
from math import log
from nltk.corpus.reader import XMLCorpusReader

In [2]:
bible = XMLCorpusReader('', 'ru_bible.xml')

In [3]:
words = bible.words()
sents = ' '.join(words)

In [4]:
st = nltk.tokenize.PunktSentenceTokenizer()
words = nltk.tokenize.word_tokenize(sents)
separated_sentenses = st.tokenize(sents)

print(len(separated_sentenses), len(words))

28407 702152


In [6]:
vocab = {}
c = 0
unique = set(words)
for i in range(len(words)):
    words[i] = words[i].lower()

for i in unique:
    for j in words:
        if i == j:
            vocab[i] = vocab.get(i, 0) + 1
    if c % 1000 == 0:
        print(f"{c} / {len(unique)}", end = '\r')
    c+=1

['в', 'начале', 'сотворил', 'бог', 'небо', 'и', 'землю', '.', 'земля', 'же']


In [7]:
dick = {}

for i in range(1, len(words)):
    probs =  (dick.get(f"{words[i-1]} {words[i]}", 0) + 1)
    dick[f"{words[i-1]} {words[i]}"] = probs
for key in dick.keys():
    logit = log(dick[key] / vocab[key.split()[0]])
    dick[key] = -logit if logit < 0 else logit
    

In [10]:
import random
n = 500
starting_word = 'бог'
ans = [starting_word]
curr_word = starting_word
for i in range(n):
    max_proba = 0
    word = ''
    for j in dick:
        if j.split()[0] == ans[-1]:
            if dick[j] > max_proba:
                max_proba = dick[j] + random.randrange(-1, 2)
                word = j.split()[1]
    ans.append(word)
    if max_proba == 0:
         break
' '.join(ans)

'бог твердь посреди воды потопа , скотов по тверди силы противостоять врагам ; сотворим человека так совершены небо ваше занятие ваше занятие ваше сделаю того колодезя был хитрее '

In [ ]:
import pickle

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [ ]:
def min_edit_distance(sourse, target):
    n = len(sourse)
    m = len(target)
    D = np.zeros((n+1, m+1), dtype = np.int64)
    
    D[:, 0] = np.cumsum(np.arange(0, n + 1))
    D[0, :] = np.cumsum(np.arange(0, m + 1))
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            D[i, j] = min(
                D[i-1, j] + 1,
                D[i, j-1] + 1,
                D[i-1, j-1] + 2 if sourse[i-1] != target[j-1] else D[i-1, j-1]
            )
    return D[-1, -1]
if __name__ == "__main__":
    print(min_edit_distance('Дима', 'Димус'))